A ternary search tree has nodes with the following attributes:
* a character, can be `None`;
* a Boolean flag that indicates whether the character represented
  by this node has been the last in a string that was inserted in the
  tree;
* the "less-than" child;
* the "equals" child and
* the "larger-than" child.

The data structure should support the following operations:
* string insert
* string search
* prefix string search
* return the number of strings stored in the data structure
* return all strings stored in the data structure

Also ensure that an instance of the data structure can be visualy represented, e.g., in aSCII format.

# Implementation

In [255]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The data structure has been implemented as a class.

In [256]:
from __future__ import (
    annotations,
)
from typing import List

class Node():
    def __init__(self, letter: str, last: bool = False):
        self._letter = letter
        self._left, self._right, self._equal = None, None, None
        self._last = last

    def __repr__(self):
        return self.letter
    
    @property
    def last(self):
        return self._last

    @last.setter
    def last(self, last: str):
        #print('setting ', last, ' as last property of the node.')
        self._last = last
    
    @property
    def letter(self):
        return self._letter

    @letter.setter
    def letter(self, letter: str):
        #print('setting ', letter, ' as letter of the node.')
        self._letter = letter


    @property
    def left(self):
        return self._left

    @left.setter
    def left(self, left: Node):
        #print('setting left neighbor of node ', self,  'to ', left)
        self._left = left

    @property
    def right(self):
        return self._right

    @right.setter
    def right(self, right):
        #print('setting right neighbor of node ', self,  'to ', right)
        self._right = right

    @property
    def equal(self):
        return self._equal

    @equal.setter
    def equal(self, equal):
        #print('setting equal neighbor of node ', self,  'to ', equal)
        self._equal = equal

    




    

In [761]:
class TernarySearchTree():
    "Ternary search tree is a tree "
    def __init__(self, root: Node = None)-> None:
        self._root = root
        
    @property
    def root(self):
        return self._root

    @root.setter
    def root(self, root):
        #print('setting root of tree ', self,  'to ', root)
        self._root = root
      
    def __repr__(self)-> str:
        """prints the tree recursively"""
        def repr_rec(current: Node, depth: int, repr_string: str = "")-> str:
            """helper function to perform the recursion"""
            if current.left:#if left node: add left node and perform depth first search for left part of the tree
                repr_string += depth*"    "+"lt: "+current.left.letter+" terminates "+str(current.left.last)+"\n"\
                + repr_rec(current.left, depth+1, "")
            if current.equal:#if equal node: add equal node and perform depth first search for equal part of the tree
                repr_string += depth*"    "+"eq: "+current.equal.letter+" terminates "+str(current.equal.last)+"\n"\
                + repr_rec(current.equal, depth+1, "")
            if current.right:#if right node: add right node and perform depth first search for right part of the tree
                repr_string += depth*"    "+"gt: "+current.right.letter+" terminates "+str(current.right.last)+"\n"\
                + repr_rec(current.right, depth+1, "")
            return repr_string
            
        if self.root:
            repr_string = "root: "+self.root.letter+"\n"+repr_rec(self.root, 1)
            return repr_string
        else:
            return "this tree is empty"
                
                
    def __len__(self) -> int:  
        """counting the number of words in the tree by countign how many nodes in the tree a last nodes"""
        def get_successessors_rec(self, node: Node, successors:List[Node] = []):
            """recursive function getting all successors of a node and returning them in a list"""
            if node:
                if not node.left and not node.right and not node.equal:
                    return []
                else:
                    if node.right:
                        successors.append(node.right)
                        successors.extend(get_successessors_rec(self, node.right, []))
                    if node.left:
                        successors.append(node.left)
                        successors.extend(get_successessors_rec(self, node.left, []))
                    if node.equal:
                        successors.append(node.equal)
                        successors.extend(get_successessors_rec(self, node.equal, []))
                return successors
            else:
                return []
        
        counter = 0
        successors = get_successessors_rec(self, self.root, [])
        successors.append(self.root)
        for node in successors:
            if node:
                if node.last == True:
                    counter += 1
        return counter
            
 
    
    def insert(self, string):
        """inserts the string into the tree"""
        if not self.root:#for an empty tree: create the root
            if string: 
                self.root = Node(string[0])
                current = self.root
                for i, letter in enumerate(string[1:]):
                    to_insert = Node(letter)
                    if i == len(string)-2:
                        to_insert.last = True
                    current.equal = to_insert
                    current = current.equal
            else:
                self.root = Node("", last = True)
        else:#otherwise insert string into exisitng tree
            current = self.root
            counter = 0
            last = False
            if string: 
                while counter < len(string):
                    if counter == len(string)-1:
                        last = True#inidcating whether we are at the last letter of the word
                    if string[counter] == current.letter:#if we find the letter in the tree
                        if last:#if it is the last letter: make this node a last node
                            current.last = True
                            counter += 1
                        elif current.equal:#otherwise: if node has equal child, move on to equal child
                            current = current.equal
                            counter += 1
                        else:#otherwise insert the next letter from the string as the new equal child
                            counter += 1
                            if counter == len(string)-1:
                                last = True
                            current.equal = Node(string[counter], last)
                            #counter += 1
                            current = current.equal
                    elif string[counter] < current.letter:#if letter of node is smaller than current letter from string: inspect left child
                        if current.left:
                            current = current.left
                        else:
                            current.left = Node(string[counter], last)#if no left child: insert letter as left child
                            current = current.left
                            #counter += 1
                    else:#if letter of node is bigger than current letter from string: inspect right child
                        if current.right:
                            current = current.right
                        else:
                            current.right = Node(string[counter], last)#if no right child: insert letter as right child
                            current = current.right
                            #counter += 1
            else:
                while current.left:
                    current = current.left
                current.left = Node("", last = True)

            
    def search(self, string:str, exact: bool = False) -> bool:
        """searching a string in the ternary tree, returns False if string not found and True if found"""
        if not self.root:#if tree empty return False
            return False
        current = self.root
        counter = 0
        while counter < len(string):
            if string[counter] == current.letter:#if we find the letter in the tree
                if current.equal and counter != len(string)-1:#and it is not the last letter of the string
                    current = current.equal#move on to equal node
                    counter += 1
                elif counter == len(string)-1:#if it last letter increase counter by 1 and pass
                    counter += 1
                else:
                    return False#if no equal child and not last letter of string: string is not in the tree
            elif string[counter] < current.letter:#if letter smaller than letter of node
                if current.left:
                    current = current.left#move on to left child
                else:
                    return False
            else:
                if current.right:
                    current = current.right
                else:
                    return False
        
        if exact == False:
            return True
        else:
            if current.last == True:
                return True
            else:
                return False
    
    def all_strings(self) -> List[str]:
        def get_all_strings_rec(current: Node, word: str):
            strings = []
            if current.left:
                strings.extend(get_all_strings_rec(current.left, word))
            if current.right: 
                 strings.extend(get_all_strings_rec(current.right, word))
            if current.equal:
                strings.extend(get_all_strings_rec(current.equal, word+current.letter))
            if current.last:
                strings.extend([word+current.letter])
            return strings

        if not self.root:
            return None
        current = self.root
        strings = get_all_strings_rec(current, "")
        return strings

                
                

In [762]:
t = TernarySearchTree()

In [763]:
len(t)

0

In [764]:
t.all_strings()

In [765]:
t.insert("hello")

In [766]:
t.all_strings()

['hello']

In [767]:
len(t)

1

In [768]:
t.insert("hel")
t.insert("abc")

In [769]:
len(t)

3

In [770]:
t.all_strings()

['abc', 'hello', 'hel']

In [771]:
t.search("helo")



False

In [772]:
t.search("hel")

True

In [773]:
t.search("hello")

True

In [774]:
t.search("hello", exact = True)

True

In [775]:
t.search("hel", exact = True)

True

In [776]:
t.search("hell")

True

In [777]:
t.search("hell", exact = True)

False

In [778]:
t.search("he", exact = True)

False

In [779]:
t.search("he")

True

In [780]:
#t.insert("")

In [781]:
len(t)

3

In [782]:
string = "a"

In [783]:
if not string:
    print(True)

In [784]:
print(t)

root: h
    lt: a terminates False
        eq: b terminates False
            eq: c terminates True
    eq: e terminates False
        eq: l terminates True
            eq: l terminates False
                eq: o terminates True



In [785]:
t.all_strings()

['abc', 'hello', 'hel']

# Example usage

Create a new empty ternery search tree.

In [261]:
tst = TernarySearchTree()

Insert the string `'abc'` into the tree.

In [262]:
tst.insert('abc')

Display the tree.

In [263]:
print(tst)

Insert another string `'aqt'`.

In [264]:
tst.insert('aqt')

In [265]:
print(tst)

The tree should now contain two strings.

In [266]:
len(tst)

2

In [267]:
tst.all_strings()

['aqt', 'abc']

Search for the string `'ab'`, it should be found since it is a prefix of `'abc'`.

In [268]:
tst.search('ab')

True

The string `'ac'` should not be found.

In [269]:
tst.search('ac')

False

The tree can also contain the empty string.

In [270]:
tst.insert('')

In [271]:
len(tst)

3

In [272]:
print(tst)

In [273]:
tst.all_strings()

['', 'aqt', 'abc']

# Testing

The file `data/search_trees/insert_words.txt` contains words that we can insert into a tree.

In [286]:
tst = TernarySearchTree()
with open('data/search_trees/insert_words.txt') as file:
    words = [
        line.strip() for line in file
    ]
for word in words:
    tst.insert(word)
unique_words = set(words)

Verify the length of the data stucture.

In [287]:
assert len(tst) == len(unique_words), \
print(f'{len(tst)} in tree, expected {len(unique_words)}')

Verify that all words that were inserted can be found.

In [288]:
for word in unique_words:
    assert tst.search(word), f'{word} not found'

Verify that all prefixes can be found.

In [289]:
for word in unique_words:
    for i in range(len(word) - 1, 0, -1):
        prefix = word[:i]
        assert tst.search(prefix), f'{prefix} not found'

Chack that when searching for a exact match, only the inserted words are found, and no prefixes.

In [290]:
for word in unique_words:
    for i in range(len(word), 0, -1):
        prefix = word[:i]
        if prefix not in unique_words:
            assert not tst.search(prefix, exact=True), \
                   f'{prefix} found'

Check that the empty string is in the tree (since it is a prefix of any string).

In [291]:
assert tst.search(''), 'empty string not found'

Check that the empty string is not in the tree for an exact search.

In [292]:
assert not tst.search('', exact=True), 'empty string found'

Check that words in the file `data/search_trees/not_insert_words.txt` can not be found in the tree.

In [293]:
with open('data/search_trees/not_insert_words.txt') as file:
    for line in file:
        word = line.strip()
        assert not tst.search(word), f'{word} should not be found'

Check that all strings are returned.

In [294]:
all_strings = tst.all_strings()
assert len(all_strings) == len(unique_words), \
       f'{len(all_strings)} words, expected {len(unique_words)}'
assert sorted(all_strings) == sorted(unique_words), 'words do not match'

If not output was generated, all tests have passed.

In [254]:
print(2*"    ","lt: ",'')

         lt:  
